In [182]:
from ruamel.yaml import YAML
import ldap
import re
import subprocess

In [183]:
conf: dict
l: int 

In [184]:
def get_conf():
    global conf, l
    with open('/Users/lilongen/cdh.security+ldap.yml', 'r') as f: conf = YAML().load(f)

def bind_ldap():
    global conf, l
    ldap_conf = conf['ldap']
    l = ldap.initialize(ldap_conf['url'])
    l.simple_bind_s(ldap_conf['bind_dn'], ldap_conf['bind_pw'])

def unbind_ldap():
    global conf, l
    l.unbind_s()

In [185]:
def get_ldap_users():
    global conf, l
    re_1st_ou = re.compile(r'CN=[^,]+,OU=([^,]+),')
    search_res = l.search(ldap_conf['base_dn'], ldap.SCOPE_SUBTREE, ldap_conf['filter'], ldap_conf['attrs'])
    users = []
    while True:
        result_type, result_data = l.result(search_res, 0)
        if (len(result_data) == 0):
            break
        result_type == ldap.RES_SEARCH_ENTRY and users.append(result_data)

    user_ids = []

    for entry in users:
        user_dn = entry[0][0]
        user_id = entry[0][1]['sAMAccountName'][0].decode()
        user_1stou = re.match(re_1st_ou, user_dn).group(1)
        user_ids.append(user_id)
    conf['role']['g_dev']['user'] = user_ids

In [186]:
def generate_group_user_directory_playbook():
    global conf, l
    playbook = []
    for r_name, r in conf['role'].items():
        playbook.append({
            'name': 'Ensure group "{}" exists'.format(r_name),
            'group': {
                'name': r_name,
                'state': 'present'
            }
        })
        playbook.append({
            'name': 'Ensure directory "{}" exists'.format(r['workspace']),
            'file': {
                'path': r['workspace'],
                'state': 'directory',
                'mode': '0755',
                'group': r_name,
            }
        })

        for u in r['user']:
            playbook.append({
                'name': 'User "{}" info'.format(u),
                'user': {
                    'name': u,
                    'state': 'present',
                    'group': r_name
                }
            })
    
    with open('/Users/lilongen/kerberos+ldap.yml', 'w') as f: YAML().dump(playbook, f)

In [186]:
def get_presence_and_yaml_diff():
    

In [187]:
def generate_keytab():
    global conf, l
    cmd_tpl = 'kadmin -p {admin} -w {admin_pw} ktadd -k {keytab_to}/{username}.keytab'
    cmds = ''
    kadmin_vars = conf['kerberos']
    for r_name, r in conf['role'].items():
        for u in r['user']:
            kadmin_vars['username'] = u
            cmds += cmd_tpl.format(**kadmin_vars) + '\n'
    print(cmds)
      

In [255]:
def get_node_user_group():
    global conf, l
    host = conf['cluster']['sample_host']
    result = subprocess.run(['ssh', 'root@{}'.format(host), "grep g_ /etc/group | awk -F: '{print $1}'"], stdout=subprocess.PIPE)
    groups = result.stdout.decode().split('\n')[0:-1]
    group_users = {}
    for g in groups:
        result = subprocess.run(['ssh', 'root@{}'.format(host), 'lid -n -g {}'.format(g)], stdout=subprocess.PIPE)
        group_users[g] = result.stdout.decode().split('\n')[0:-1]
    
    conf['presence_role'] = group_users

In [256]:
def main():
    get_conf()
    print(conf)
    bind_ldap()
    get_ldap_users()
    get_node_user_group()
    generate_group_user_directory_playbook()
    generate_keytab()
    get_node_user_group()
    unbind_ldap()


In [257]:
main()

ordereddict([('ldap', ordereddict([('url', 'ldap://221.224.33.71:389/'), ('intra_url', 'ldap://172.17.100.10:389/'), ('bind_dn', 'tqy@domain.oceansoft'), ('bind_pw', 'oceanitg2016!@#'), ('base_dn', 'OU=云学堂,dc=domain,dc=oceansoft'), ('filter', '(&(objectCategory=person)(objectClass=user)(memberOf=cn=BDAI,OU=组,dc=domain,dc=oceansoft))'), ('attrs', ['sAMAccountName', 'mail', 'mobile']), ('filter1', '(&(objectCategory=person)(objectClass=user)(|(department=大数据&DevOps)(department=南京研发团队)))'), ('attrs1', ['sAMAccountName', 'mail', 'mobile'])])), ('role', ordereddict([('g_dev', ordereddict([('user', None), ('workspace', '/wsdev')])), ('g_prod', ordereddict([('user', ['prt', 'pol']), ('workspace', '/wsprod')]))])), ('kerberos', ordereddict([('admin', 'root/admin'), ('admin_pw', 'Uydc.2910'), ('keytab_to', '/tmp')])), ('cluster', ordereddict([('sample_host', '10.200.70.101'), ('user', 'root')]))])
kadmin -p root/admin -w Uydc.2910 ktadd -k /tmp/lile.keytab
kadmin -p root/admin -w Uydc.2910 ktad